In [1]:
import pandas as pd
import numpy as np
import random
import networkx as nx
import bridgeness
from numpy.random import choice, uniform

In [2]:
FN = "graph.txt.csv"
data = pd.read_csv(FN, delimiter = " ")

G = nx.Graph()

for i,s in data.iterrows():
    u = s[0]
    v = s[1]

    G.add_edge(u,v)

In [3]:
bri = bridgeness.bridgeness_centrality(G)
#e_bri = bridgeness.edge_bridgeness_centrality(G)
#bet = bridgeness.betweenness_centrality(G)

In [7]:
phi = 100
mu = 0.2
lab0 = 95
lab1 = 190

adj = nx.to_numpy_matrix(G)
bri_vals = list(bri.values())
quality = (1 / np.exp( np.multiply( bri_vals, phi )))
diag = np.diagflat(quality) #k: v = node, bridgeness
_tmp = np.matmul(diag, adj)
_tmp_sum_on_rows = np.sum(_tmp, axis=0)
_tmp_sum_on_rows_recip = np.reciprocal(_tmp_sum_on_rows)
norm = np.diagflat( _tmp_sum_on_rows_recip )

T = np.matmul( _tmp, norm )

num_nodes = len(G.nodes())
alpha = 2.0/(2.0 + mu)
prob_comm = T 

labels = np.zeros( (num_nodes, 2) )
labels[lab0,0] = 1
labels[lab1,1] = 1

op1 = (1-alpha) * np.eye(num_nodes) 
op2 = ( np.eye(num_nodes) - ( alpha * T))
op = np.dot(op1, op2.I)

probs = np.dot(op, labels)

In [8]:
#pd.DataFrame.to_csv(pd.DataFrame(probs), 'probs.csv')

In [ ]:
#2-comm RW

#start from lab0(s), lab1(s)

steps = 1000000 #max steps
convcheckfreq = 10000 #check if converged every N steps
conv_thr = 1e-06

n0 = lab0
n1 = lab1

labels=[n0,n1]

#visit probability vec
rw_visits = np.zeros( (num_nodes, 2) )
last_rw_visits = np.copy(rw_visits)

nodes = range(num_nodes)

print("Alpha = %.3f" %(alpha))

#COMM0
for s0 in range(1,steps+1):

    #start from
    #go back to labeled node? prob = 1-alpha; prob to trans = alpha
    if (uniform() < alpha): #
        trans_probs = T[:,n0].view(np.ndarray).flatten() 
        trans_to = int( choice(nodes, 1, p = trans_probs) )
        n0 = trans_to
    else:
        n0 = lab0

    rw_visits[n0,0] += 1
    
    if (s0 % convcheckfreq == 0):
        diff_rw_visits = ((rw_visits[:,0]/s0 - last_rw_visits[:,0]/s0)**2).sum()
        print("At step %d, diff %.10f" %(s0, diff_rw_visits) )
        if ((diff_rw_visits) < conv_thr):# and np.all(rw_visits != last_rw_visits)):
            print("Converged at <%.10f" % conv_thr)
            break
        last_rw_visits = np.copy(rw_visits)

#COMM1
for s1 in range(1,steps+1):

    #start from
    #go back to labeled node? prob = 1-alpha; prob to trans = alpha
    if (uniform() < alpha):
        trans_probs = T[:,n1].view(np.ndarray).flatten() 
        trans_to = int( choice(nodes, 1, p = trans_probs) )
        n1 = trans_to
    else:
        n1 = lab1

    rw_visits[n1,1] += 1
    
    if (s1 % convcheckfreq == 0):
        diff_rw_visits = ((rw_visits[:,1]/s1 - last_rw_visits[:,1]/s1)**2).sum()
        print("At step %d, diff %.10f" %(s1, diff_rw_visits) )
        if ((diff_rw_visits) < conv_thr):# and np.all(rw_visits != last_rw_visits)):
            print("Converged at <%.10f" % conv_thr)
            break
        last_rw_visits = np.copy(rw_visits)


Alpha = 0.909
At step 10000, diff 0.0308368200
At step 20000, diff 0.0074185050
At step 30000, diff 0.0033096111


In [ ]:
#comm_cf = ( (probs[:,0]) > (probs[:,1]) )
#comm_rw = ( (rw_visits[:,0]/s0) > (rw_visits[:,1]/s1) )
#print(comm_cf, comm_rw)
#print((rw_visits[:,0]/s0, rw_visits[:,1]/s0) )